# **Set-up**


In [1]:
%%capture
!pip install -r requirements

In [2]:
# Import Packages
## for data and preprocessing
import pandas as pd
from sklearn.preprocessing import StandardScaler

## for model fitting
import lightgbm as lgb
import sklearn.metrics as metric

## for hyperparameter optimization
import optuna

## for replicability
import random

In [3]:
train = pd.read_csv('./data/california_housing_train.csv')
test = pd.read_csv('./data/california_housing_test.csv')

names = train.columns

scaler = StandardScaler()
train = pd.DataFrame(scaler.fit_transform(train),columns=names)
test = pd.DataFrame(scaler.transform(test), columns=names)


X_train = train.drop(['median_house_value'],axis=1)
X_test  = test.drop(['median_house_value'],axis=1)
y_train = train.median_house_value
y_test  = test.median_house_value

# **OPTUNA**

## **General Overview**

Optuna optimizes any objective function. This objective function takes a set of arguments (e.g., hyperparameters) and returns a single value (e.g., validation score).  

In Optuna, we create a **study**. A study is defined by the objective function and the hyperparameter space and, thus, defines the scope and purpose of our optimization exercise.   
Each study consists of a set of **Trials**. Each trial is, thus, a single selection from the hyperparameter space for which we evaluate the objective function. Every next trial builds on the previous one (i.e., an iterative optimization process).

The optimization algorithm helps in picking the next trial to evaluate in a smart(er) way, until we find the optimal value.

In practice, every hyperparameter optimization exercise consist of 4 steps:

* define a function which **trains a model** and **returns the validation score**

* define the **hyperparameter space** through which the optimization algorithm can search (trials are instances/realizations of this space)

* create a **study**, which describes the optimization exercise: 
    * *Direction* : 
        * minimize: for (Root) Mean Squared Errors, minus-log-likelihood, ... (the lower, the better)
        * maximize: r2_score, auc, accuracy, precision, recall, f1_score, ... (the higher, the better)
    * *Sampler* : the chosen optimization technique **(Optimization)**
    * *Pruner* : early stopping of unpromising trials **(Steroids)**

* **optimize** the study using different trials in a smart way **(worker function)**


Firstly, we need to realize that our time is also limited. In order to limit our waiting time (and computing time), we set a maximum number of trials to evaluate (i.e., maximum number of iterations). 

In [4]:
N_TRIALS = 200

### Step 1
We define a function which takes a hyperparameter configuration (params;  which is defined later) as the argument.  
Then this function takes our data and trains a machine learning model. In this example, we train a lightgbm model, which can take a lot of interesting hyperparameters to illustrate tuning. Any model architecture can work here (e.g., xgboost, random forest, neural networks, ...).  
Lastly, we make some predictions on our test (or validation) set and compute the validation score. In this example, we use the Root Mean Squared Error (RMSE), but again any validation metric is viable.

In [5]:
def train_evaluate(params):
    '''Train a model using your dataset and return the validation score.'''
    train_data = lgb.Dataset(X_train, label=y_train)
    test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)
    # Train a Model
    model = lgb.train(params, train_data,
                      num_boost_round=params['NUM_BOOST_ROUND'],
                      early_stopping_rounds=params['EARLY_STOPPING_ROUNDS'],
                      valid_sets=[test_data],
                      valid_names=['valid'],
                      )
    # Evaluate the model
    preds = model.predict(test_data,num_iteration=model.best_iteration)
    truth = test_data.get_label()
    score = metric.mean_squared_error(truth, preds, squared=False)
      
    #score = model.best_score['valid']['rmse']
    # Return the validation score
    return score

### Step 2
We define the objective function of our optimization exercise, which takes a trial as argument.  
In this function, we first define the parameter space. This parameter space is a dictionary defining each hyperparameter of interest. For each hyperparameter, we use the $trial.suggest$ functionality to define the domain from which we can sample values for the hyperparameters. In a Bayesian way, think about this as our prior (hyper)parameter distribution. We can use various distributions:
- trial.suggest_loguniform for floating point hyperparameters between two bounds favoring smaller values,
- trial.suggest_float for floating point hyperparameters between two bounds
- trial.suggest_int for integer hyperparameters between two bounds and a step-size,
- trial.suggest_uniform for uniformly distributed hyperparameters between two bounds,
- trial.suggest_discrete_uniform for uniformly distrubuted hyperparameters between two bounds but with additional step-size,
- ...

After defining the hyperparameter space, we apply the previously defined function to train a model and return the validation score.  
At this stage, we will also check whether the score should be pruned or not (depending on whether a pruning strategy was specified).

In [6]:
def objective(trial):
    '''
    Define the Hyperparameter Space from which to sample a configuration.
    Then train a model and output the validation score (see Step 1).
    '''
    # Define the Hyper-parameter Space
    params = {'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
              'max_depth': trial.suggest_int('max_depth', 1, 30, 1),
              'num_leaves': trial.suggest_int('num_leaves', 2, 100),
              'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
              'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 1.0),
              'subsample': trial.suggest_discrete_uniform('subsample', 0.1, 1.0,.1),
              'colsample_by_tree': 1,
              'lambda_l1': trial.suggest_float('lambda_l1', 0, 10),
              'lambda_l2': trial.suggest_float('lambda_l2', 0, 10),
              'NUM_BOOST_ROUND': 200,
              'EARLY_STOPPING_ROUNDS': 20,
              'objective': 'rmse',
              }
              
    # Train the model and return the validation score
    score = train_evaluate(params)
    
    #Check Pruning
    trial.report(score,1)
    if trial.should_prune():
        raise optuna.TrialPruned()

    # Return the validation score
    return score

### Step 3
We create the study object, in which we describe the optimization exercise by means of: 

* the *Direction* of our optimization: 
  * minimize: for (Root) Mean Squared Errors, minus-log-likelihood, ... (the lower, the better)
  * maximize: r2_score, auc, accuracy, precision, recall, f1_score, ... (the higher, the better)  
  

* the *Sampler* which is our optimization technique: 
  * GridSampler, applies a Grid Search on a predefined grid (extra arguments required!)
  * RandomSampler, applies Random Search on the parameter space
  * CmaEsSampler, applies a Covariance Matrix Adaptation Evolutionary Search algorithm
  * TPESampler, is the default option, which applies a Tree-structured Parzen Estimator algorithm  


* the *Pruner* which is our pruning strategy to quickly stop unpromising trials:
  * NopPruner, does not prune any trials
  * MedianPruner, prunes trials that are worst than the median of previous trials
  * SuccessiveHalvingPruner, uses Asynchronous Successive Halving (prune half of the least performing trials)
  * HyperbandPruner, uses the Hyperband pruning strategy  

In [7]:
study = optuna.create_study(
    direction = 'minimize',                         
    sampler = optuna.samplers.RandomSampler(),      
    pruner = optuna.pruners.NopPruner()            
    )

[I 2021-11-08 17:38:41,247] A new study created in memory with name: no-name-ce3609f8-03ce-47d6-9852-6c7e4687a1e5


### Step 4
We call the optimize function on our study object to start the optimization process. 

In [8]:
%%script false --no-raise-error
study.optimize(objective, n_trials=N_trials)

Couldn't find program: 'false'


## Example of Several Optimization Strategies

Uncomment the following line if you want to suppress all output of the optuna sampler.

In [9]:
#optuna.logging.set_verbosity(optuna.logging.WARNING)

In [10]:
random.seed(1)

Next, we define out $train\_evaluate$ and our $objective$ functions. 

In [11]:
def train_evaluate(params):
    # Format/Preprocess Data
    train_data = lgb.Dataset(X_train, label=y_train)
    test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)
    
    # Train a Model
    model = lgb.train(params, train_data,
                      num_boost_round=params['NUM_BOOST_ROUND'],
                      early_stopping_rounds=params['EARLY_STOPPING_ROUNDS'],
                      valid_sets=[test_data],
                      valid_names=['valid'],
                      )
    
    # Evaluate the model 
    preds = model.predict(X_test,num_iteration=model.best_iteration)
    truth = test_data.get_label()
    score = metric.mean_squared_error(truth, preds, squared=False)
    
    # Return the validation score
    return score

def objective(trial):
    # Define the Hyper-parameter Space
    params = {'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
              'max_depth': trial.suggest_int('max_depth', 1, 50),
              'num_leaves': trial.suggest_int('num_leaves', 2, 200),
              'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 1.0),
              'subsample': trial.suggest_discrete_uniform('subsample', 0.1, 1.0, .1),
              'colsample_by_tree': 1,
              'lambda_l1': trial.suggest_float('lambda_l1', 0, 10),
              'lambda_l2': trial.suggest_float('lambda_l2', 0, 10),
              'bagging_fraction':trial.suggest_uniform('bagging_fraction', 0, 1),
              'bagging_freq':trial.suggest_int('bagging_freq',0,10),
              'NUM_BOOST_ROUND': 200,
              'EARLY_STOPPING_ROUNDS': 20,
              'objective': 'rmse',
              }
    
    # Train the model and return the validation score
    score = train_evaluate(params)
    
    #Check Pruning
    trial.report(score,200)
    if trial.should_prune():
      raise optuna.TrialPruned()
    
    # Return the validation score
    return score

### **Grid Search**

A grid search does not really look at the hyperparameter space, but rather takes a search space with discrete lists of hyperparameter values into account.
In this example, we search over a small grid of 4*4*4 hyperparameters (total size of the grid: 48 possibilities).

In [12]:
%%time
%%capture

def objective_grid(trial):
    # Define the Hyper-parameter Space
    params = {'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5),
              'max_depth': trial.suggest_int('max_depth', 1, 50),
              'num_leaves': trial.suggest_int('num_leaves', 2, 200),
              'NUM_BOOST_ROUND': 200,
              'EARLY_STOPPING_ROUNDS': 20,
              'objective': 'rmse',
              'verbose': -1,
              }
    #Apply the train_evaluate function
    score = train_evaluate(params)
    return score

search_space = {'learning_rate': [0.01, 0.10, 0.50],
              'max_depth': [1, 10, 20, 30],
              'num_leaves': [2, 10, 20, 100]}

study_gridsearch = optuna.create_study(
    direction='minimize',
    sampler=optuna.samplers.GridSampler(search_space),
    pruner = optuna.pruners.NopPruner() 
    )

study_gridsearch.optimize(objective_grid, n_trials=N_TRIALS)

[I 2021-11-08 17:38:41,436] A new study created in memory with name: no-name-0a3509d8-9c41-4ede-970e-e9b822b9b0cb
[I 2021-11-08 17:38:41,504] Trial 0 finished with value: 0.7470006743845352 and parameters: {'learning_rate': 0.01, 'max_depth': 20, 'num_leaves': 2}. Best is trial 0 with value: 0.7470006743845352.
[I 2021-11-08 17:38:41,663] Trial 1 finished with value: 0.42793408300698016 and parameters: {'learning_rate': 0.5, 'max_depth': 10, 'num_leaves': 10}. Best is trial 1 with value: 0.42793408300698016.
[I 2021-11-08 17:38:41,754] Trial 2 finished with value: 0.7470006743845352 and parameters: {'learning_rate': 0.01, 'max_depth': 10, 'num_leaves': 2}. Best is trial 1 with value: 0.42793408300698016.
[I 2021-11-08 17:38:41,837] Trial 3 finished with value: 0.5988083376612268 and parameters: {'learning_rate': 0.1, 'max_depth': 1, 'num_leaves': 20}. Best is trial 1 with value: 0.42793408300698016.
[I 2021-11-08 17:38:41,968] Trial 4 finished with value: 0.5943733156686446 and paramet

Wall time: 7.2 s


In [13]:
gridsearch = {'score': study_gridsearch.best_value, 'params': study_gridsearch.best_params}
print(gridsearch)

{'score': 0.4009600440564793, 'params': {'learning_rate': 0.1, 'max_depth': 10, 'num_leaves': 100}}


### **Random Search**

In [14]:
%%time
%%capture

study_randomsearch = optuna.create_study(
    direction = 'minimize',
    sampler = optuna.samplers.RandomSampler(),
    pruner = optuna.pruners.NopPruner() 
    )

study_randomsearch.optimize(objective, n_trials=N_TRIALS)

[I 2021-11-08 17:38:48,723] A new study created in memory with name: no-name-38c8cf09-90b0-4952-b769-65d2788c6f99
[I 2021-11-08 17:38:49,044] Trial 0 finished with value: 0.4778734311745048 and parameters: {'learning_rate': 0.12704180938770063, 'max_depth': 24, 'num_leaves': 21, 'feature_fraction': 0.1889242485048997, 'subsample': 0.4, 'lambda_l1': 3.4268989036758635, 'lambda_l2': 7.6163051425424975, 'bagging_fraction': 0.3768619943287135, 'bagging_freq': 6}. Best is trial 0 with value: 0.4778734311745048.
[I 2021-11-08 17:38:49,614] Trial 1 finished with value: 0.4089903580603225 and parameters: {'learning_rate': 0.028872007939517635, 'max_depth': 31, 'num_leaves': 119, 'feature_fraction': 0.7459935462740664, 'subsample': 0.6, 'lambda_l1': 1.6098537795466217, 'lambda_l2': 3.252387295111655, 'bagging_fraction': 0.39599847569663116, 'bagging_freq': 3}. Best is trial 1 with value: 0.4089903580603225.
[I 2021-11-08 17:38:49,691] Trial 2 finished with value: 0.5819267260023515 and paramete

Wall time: 1min 29s


In [15]:
randomsearch = {'score': study_randomsearch.best_value, 'params': study_randomsearch.best_params}
print(randomsearch)

{'score': 0.39651979611630067, 'params': {'learning_rate': 0.04546437531253444, 'max_depth': 13, 'num_leaves': 200, 'feature_fraction': 0.58398483825196, 'subsample': 0.30000000000000004, 'lambda_l1': 3.3659279839214973, 'lambda_l2': 3.0583347511218193, 'bagging_fraction': 0.9139336052567875, 'bagging_freq': 0}}


### **CMAES**

In [16]:
%%time
%%capture

study_cmaes = optuna.create_study(
    direction = 'minimize',
    sampler = optuna.samplers.CmaEsSampler(),
    pruner = optuna.pruners.MedianPruner()  
    )

study_cmaes.optimize(objective, n_trials=N_TRIALS)

[I 2021-11-08 17:40:17,895] A new study created in memory with name: no-name-d76590d9-182a-493e-89cd-c467c5d65d95
[I 2021-11-08 17:40:18,392] Trial 0 finished with value: 0.40294164221960865 and parameters: {'learning_rate': 0.10744358111460002, 'max_depth': 21, 'num_leaves': 47, 'feature_fraction': 0.7510785316103546, 'subsample': 0.4, 'lambda_l1': 5.101745808458499, 'lambda_l2': 0.4305358779428836, 'bagging_fraction': 0.8512966595533874, 'bagging_freq': 4}. Best is trial 0 with value: 0.40294164221960865.
[I 2021-11-08 17:40:19,056] Trial 1 finished with value: 0.4137684858852812 and parameters: {'learning_rate': 0.0672011132343066, 'max_depth': 25, 'num_leaves': 101, 'feature_fraction': 0.6255071740360991, 'subsample': 0.30000000000000004, 'lambda_l1': 5.190605714406583, 'lambda_l2': 5.06152342587859, 'bagging_fraction': 0.4239778127350932, 'bagging_freq': 5}. Best is trial 0 with value: 0.40294164221960865.
[I 2021-11-08 17:40:19,747] Trial 2 finished with value: 0.4104620341517344

Wall time: 2min 34s


In [17]:
cmaessearch = {'score': study_cmaes.best_value, 'params': study_cmaes.best_params}
print(cmaessearch)

{'score': 0.39534373804008377, 'params': {'learning_rate': 0.09217041850161034, 'max_depth': 25, 'num_leaves': 101, 'feature_fraction': 0.6319213954274244, 'subsample': 0.6, 'lambda_l1': 4.54613539405008, 'lambda_l2': 4.869960247357709, 'bagging_fraction': 0.930488473355766, 'bagging_freq': 5}}


### **Tree-Parzen Estimator**

In [18]:
%%time
%%capture

study_tpe = optuna.create_study(
    direction = 'minimize',
    sampler = optuna.samplers.TPESampler(),
    pruner = optuna.pruners.NopPruner()
    )

study_tpe.optimize(objective, n_trials=N_TRIALS)

[I 2021-11-08 17:42:52,214] A new study created in memory with name: no-name-4f12b512-f3ef-499c-90fa-c99825dee606
[I 2021-11-08 17:42:53,223] Trial 0 finished with value: 0.39777958750773185 and parameters: {'learning_rate': 0.05334609012206734, 'max_depth': 19, 'num_leaves': 144, 'feature_fraction': 0.8195622686545379, 'subsample': 0.6, 'lambda_l1': 3.1198031544877125, 'lambda_l2': 4.043844488143719, 'bagging_fraction': 0.8908158072211779, 'bagging_freq': 0}. Best is trial 0 with value: 0.39777958750773185.
[I 2021-11-08 17:42:53,910] Trial 1 finished with value: 0.4358752291970234 and parameters: {'learning_rate': 0.02055104060171125, 'max_depth': 27, 'num_leaves': 73, 'feature_fraction': 0.9969228057154212, 'subsample': 0.8, 'lambda_l1': 6.21174486221821, 'lambda_l2': 3.290085828953492, 'bagging_fraction': 0.6488878021188779, 'bagging_freq': 2}. Best is trial 0 with value: 0.39777958750773185.
[I 2021-11-08 17:42:54,141] Trial 2 finished with value: 0.5203807777909708 and parameters

Wall time: 3min 12s


In [19]:
tpesearch = {'score': study_tpe.best_value, 'params': study_tpe.best_params}
print(tpesearch)

{'score': 0.39038612605190603, 'params': {'learning_rate': 0.03602755861946449, 'max_depth': 45, 'num_leaves': 186, 'feature_fraction': 0.6014689332932478, 'subsample': 0.30000000000000004, 'lambda_l1': 0.32856552212928725, 'lambda_l2': 1.0803859751978806, 'bagging_fraction': 0.8185072443269765, 'bagging_freq': 3}}


### **BOHB**

Note that for BOHB, increasing the number of trials is highly beneficiary to the final result. This is, in general, true for every use-case in which Hyperband Pruning is applied.  
As the hyperband pruning algorithms applies Successive Halving on multiple sets to balance resource distributions, a higher number of trials will work better. 

In [20]:
%%time
%%capture

study_bohb = optuna.create_study(
    direction = 'minimize',
    sampler = optuna.samplers.TPESampler(),
    pruner = optuna.pruners.HyperbandPruner()
    )

study_bohb.optimize(objective, n_trials=N_TRIALS*2)

[I 2021-11-08 17:46:05,019] A new study created in memory with name: no-name-c61f9435-515d-43f3-8c40-10c21cb652f2
[I 2021-11-08 17:46:05,526] Trial 0 finished with value: 0.4841883832862934 and parameters: {'learning_rate': 0.022586424291801393, 'max_depth': 47, 'num_leaves': 81, 'feature_fraction': 0.450251502812666, 'subsample': 0.7000000000000001, 'lambda_l1': 8.900256077480686, 'lambda_l2': 0.38093164022530046, 'bagging_fraction': 0.5170134268781407, 'bagging_freq': 9}. Best is trial 0 with value: 0.4841883832862934.
[I 2021-11-08 17:46:06,202] Trial 1 finished with value: 0.6381090661203797 and parameters: {'learning_rate': 0.01471323834845577, 'max_depth': 31, 'num_leaves': 162, 'feature_fraction': 0.2560456544305305, 'subsample': 0.5, 'lambda_l1': 1.1752649644970603, 'lambda_l2': 4.8498260065316, 'bagging_fraction': 0.26861657550972107, 'bagging_freq': 1}. Best is trial 0 with value: 0.4841883832862934.
[I 2021-11-08 17:46:06,484] Trial 2 finished with value: 0.6454050071551597 

Wall time: 5min 56s


In [21]:
bohbsearch = {'score': study_bohb.best_value, 'params': study_bohb.best_params}
print(bohbsearch)

{'score': 0.39114128987333147, 'params': {'learning_rate': 0.06976579246623671, 'max_depth': 18, 'num_leaves': 165, 'feature_fraction': 0.5769590702836126, 'subsample': 0.2, 'lambda_l1': 1.7041120100896419, 'lambda_l2': 3.721305715564068, 'bagging_fraction': 0.906216278754276, 'bagging_freq': 3}}


### **Summary**
The final overview of our results shows that TPE and BOHB perform far better than the other algorithms. 
In this example, the differences are not extreme, which is mostly due to the very stylized example. In many other cases, the gains of hyperparameter optimization are considerable.

In [22]:
pd.DataFrame([gridsearch['score'],randomsearch['score'],cmaessearch['score'],tpesearch['score'],bohbsearch['score']],index=['Grid','Random','CMAES','TPE','BOHB'],columns=['RMSE'])

,RMSE
Grid,0.400960
Random,0.396520
CMAES,0.395344
TPE,0.390386
BOHB,0.391141


### **Visualization**
#### History

In [23]:
trials_df = study_bohb.trials_dataframe()
trials_df

,number,value,datetime_start,datetime_complete,duration,params_bagging_fraction,params_bagging_freq,params_feature_fraction,params_lambda_l1,params_lambda_l2,params_learning_rate,params_max_depth,params_num_leaves,params_subsample,system_attrs_completed_rung_0,system_attrs_completed_rung_1,system_attrs_completed_rung_2,system_attrs_completed_rung_3,system_attrs_completed_rung_4,state
0,0,0.484188,2021-11-08 17:46:05.020412,2021-11-08 17:46:05.526556,0 days 00:00:00.506144,0.517013,9,0.450252,8.900256,0.380932,0.022586,47,81,0.7,NaN,NaN,NaN,NaN,NaN,COMPLETE
1,1,0.638109,2021-11-08 17:46:05.526556,2021-11-08 17:46:06.202228,0 days 00:00:00.675672,0.268617,1,0.256046,1.175265,4.849826,0.014713,31,162,0.5,0.638109,0.638109,0.638109,0.638109,NaN,COMPLETE
2,2,0.645405,2021-11-08 17:46:06.202228,2021-11-08 17:46:06.484389,0 days 00:00:00.282161,0.662681,10,0.300802,0.860606,4.535342,0.014625,6,191,0.9,0.645405,0.645405,0.645405,0.645405,0.645405,COMPLETE
3,3,0.613360,2021-11-08 17:46:06.484389,2021-11-08 17:46:06.756240,0 days 00:00:00.271851,0.949791,1,0.173212,8.415879,5.046355,0.035016,31,58,1.0,0.613360,0.613360,0.613360,0.613360,0.613360,COMPLETE
4,4,0.625465,2021-11-08 17:46:06.764264,2021-11-08 17:46:07.612487,0 days 00:00:00.848223,0.438137,10,0.254125,2.570487,2.399111,0.015535,28,188,0.7,0.625465,0.625465,NaN,NaN,NaN,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,395,0.423763,2021-11-08 17:51:56.498623,2021-11-08 17:51:57.206974,0 days 00:00:00.708351,0.906071,3,0.554998,9.801587,3.329278,0.073386,18,165,0.2,0.423763,NaN,NaN,NaN,NaN,PRUNED
396,396,0.489054,2021-11-08 17:51:57.207973,2021-11-08 17:51:58.227605,0 days 00:00:01.019632,0.707287,2,0.236664,1.665786,3.918632,0.077225,33,164,0.2,0.489054,NaN,NaN,NaN,NaN,PRUNED
397,397,0.398724,2021-11-08 17:51:58.227605,2021-11-08 17:51:59.311321,0 days 00:00:01.083716,0.968729,3,0.604341,5.239395,3.498675,0.066325,18,162,0.2,0.398724,NaN,NaN,NaN,NaN,PRUNED
398,398,0.396046,2021-11-08 17:51:59.311321,2021-11-08 17:52:00.502426,0 days 00:00:01.191105,0.963281,1,0.834216,1.059878,2.260559,0.063142,30,181,0.3,0.396046,0.396046,0.396046,NaN,NaN,COMPLETE


We can see that the optimization history plot for the BOHB shows a decreasing pattern. 
In the earlier trials, the algorithm makes big leaps forward in its performance (after about 60 trials, the model has already surpassed random search).    
After that, there is a long stretch where the model barely improves.

We can also notice that the optimization results become increasingly less variable. This is due to the pruning strategy. There are far less big jumps in the model performance and the model results are close together.

In [24]:
optuna.visualization.plot_optimization_history(study_bohb)

#### Hyperparameter Importance Plot
In this plot, we can see the importance of our hyperparameters. It seems that feature fraction, the learning rate and the bagging fraction are (in order of importance) the most important hyperparameters for our model. 
The max depth, subsamples and penalization terms are, respectively, far less important to tune in our example. 

This plot might help us to more efficiently tune hyperparameters in the future (as some hyperparameters do not affect the model performance as much as others).

In [25]:
optuna.visualization.plot_param_importances(study_bohb)

#### Exploration - Exploitation Plot
In this plot, we can see which regions of each hyperparameter have been explored (on the horizontal axes). We also notice that for many important hyperparameters, the darker values (later trials) are clustered together. This means that our hyperparameter tuning exercise has reached quite good convergence on the most optimal hyperparameter value.

In [26]:
optuna.visualization.plot_slice(study_bohb)

#### Dominant Profile Plot
The last plot shows all different hyperparameter profiles. This plot becomes increasingly less useful if the number of trials increases (and even more so when no pruning strategies are used). 
However, we can clearly see on this plot that there is a 'dominant profile' which leads to the best performance metric (see the darkest profiles corresponds to the better models). 

In [27]:
optuna.visualization.plot_parallel_coordinate(study_bohb)